### 1. IMPORTS & CONFIG

In [7]:
# === 1. IMPORTS & CONFIG ===
import pandas as pd
import numpy as np
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report

# Config
SEED = 80085
N_FOLDS = 5
TARGET_ORDER = ['None_Existent', 'Low', 'Medium', 'High', 'Very_High']

print("Libraries loaded. Config set.")


Libraries loaded. Config set.


### 2. LOAD DATA

In [11]:
print("Loading Hybrid Data...")

# Load Features (X)
# Note: This file (training_data_hybrid.csv) ALREADY has the target dropped!
df_train = pd.read_csv('../data/training_data_hybrid.csv')
X = df_train  # No need to drop 'AVERAGE_SPEED_DIFF', it's not there!

# Load Target (y) from its separate file
y_df = pd.read_csv('../data/y_train_hybrid.csv')
y = y_df['AVERAGE_SPEED_DIFF'] # Extract the series

# Load Test
df_test = pd.read_csv('../data/test_data_hybrid.csv')
X_test = df_test.copy()

# Align Test columns (Just a safety check, they should match already)
X_test = X_test[X.columns] 

print(f"Data Loaded:")
print(f"  Train (X) shape: {X.shape}")
print(f"  Target (y) shape: {y.shape}")
print(f"  Test (X) shape:  {X_test.shape}")

Loading Hybrid Data...
Data Loaded:
  Train (X) shape: (6812, 23)
  Target (y) shape: (6812,)
  Test (X) shape:  (1500, 23)


### 3. TRAINING ENGINE

In [12]:
def run_cv_model(model, X, y, X_test, model_name="Model"):
    cv = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)
    
    oof_preds = np.zeros(len(y))
    test_probs_fold_sum = np.zeros((len(X_test), 5)) # 5 classes
    
    print(f"\nTraining {model_name}...")
    
    for fold, (train_idx, val_idx) in enumerate(cv.split(X, y), 1):
        X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_val = y[train_idx], y[val_idx]
        
        model.fit(X_tr, y_tr)
        
        # Validation (OOF)
        val_pred = model.predict(X_val)
        oof_preds[val_idx] = val_pred
        
        # Test (Probabilities)
        test_probs_fold_sum += model.predict_proba(X_test)
        
        acc = accuracy_score(y_val, val_pred)
        # print(f"  - Fold {fold}: {acc:.4f}") # Uncomment for detail
    
    overall_acc = accuracy_score(y, oof_preds)
    avg_test_probs = test_probs_fold_sum / N_FOLDS
    
    print(f"✨ {model_name} Overall OOF Accuracy: {overall_acc:.4f}")
    
    return oof_preds, avg_test_probs, overall_acc


### 4. HYPERPARAMETER TUNING

In [13]:
# --- Helper Function for Tuning ---
def tune_model(model, param_grid, name):
    print(f"\n🔍 Tuning {name}...")
    search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_grid,
        n_iter=10,         # 5 combos per model
        scoring='accuracy',
        cv=5,              # 5-fold 
        verbose=1,
        random_state=SEED,
        n_jobs=-1
    )
    search.fit(X, y)
    print(f"   Best {name} Params: {search.best_params_}")
    print(f"   Best {name} Score: {search.best_score_:.4f}")
    return search.best_estimator_

# --- A. XGBoost Grids ---
xgb_params = {
    'n_estimators': [300, 500],
    'max_depth': [4, 6, 8],
    'learning_rate': [0.03, 0.05, 0.1],
    'subsample': [0.7, 0.8],
    'colsample_bytree': [0.7, 0.8],
    'min_child_weight': [1, 3]
}
best_xgb = tune_model(
    xgb.XGBClassifier(objective='multi:softprob', random_state=SEED, n_jobs=-1),
    xgb_params, "XGBoost"
)

# --- B. Random Forest Grids ---
rf_params = {
    'n_estimators': [300, 500],
    'max_depth': [10, 15, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}
best_rf = tune_model(
    RandomForestClassifier(random_state=SEED, n_jobs=-1),
    rf_params, "RandomForest"
)

# --- C. LightGBM Grids ---
lgb_params = {
    'n_estimators': [300, 500],
    'learning_rate': [0.03, 0.05, 0.1],
    'num_leaves': [20, 31, 50],       # Important for LGBM!
    'max_depth': [-1, 10, 20],
    'subsample': [0.7, 0.8],
    'colsample_bytree': [0.7, 0.8]
}
best_lgb = tune_model(
    lgb.LGBMClassifier(random_state=SEED, n_jobs=-1, verbose=-1),
    lgb_params, "LightGBM"
)



🔍 Tuning XGBoost...
Fitting 5 folds for each of 10 candidates, totalling 50 fits
   Best XGBoost Params: {'subsample': 0.7, 'n_estimators': 300, 'min_child_weight': 3, 'max_depth': 4, 'learning_rate': 0.03, 'colsample_bytree': 0.7}
   Best XGBoost Score: 0.7977

🔍 Tuning RandomForest...
Fitting 5 folds for each of 10 candidates, totalling 50 fits
   Best RandomForest Params: {'n_estimators': 500, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 15}
   Best RandomForest Score: 0.7908

🔍 Tuning LightGBM...
Fitting 5 folds for each of 10 candidates, totalling 50 fits
   Best LightGBM Params: {'subsample': 0.7, 'num_leaves': 50, 'n_estimators': 300, 'max_depth': 10, 'learning_rate': 0.03, 'colsample_bytree': 0.8}
   Best LightGBM Score: 0.7910


### 5. TRAIN & EVALUATE BEST MODELS

In [14]:
print("\nTraining Final Models with Best Parameters...")

# A. XGBoost
oof_xgb, probs_xgb, acc_xgb = run_cv_model(best_xgb, X, y, X_test, "XGBoost (Tuned)")

# B. Random Forest
oof_rf, probs_rf, acc_rf = run_cv_model(best_rf, X, y, X_test, "RandomForest (Tuned)")

# C. LightGBM
oof_lgb, probs_lgb, acc_lgb = run_cv_model(best_lgb, X, y, X_test, "LightGBM (Tuned)")



Training Final Models with Best Parameters...

Training XGBoost (Tuned)...
✨ XGBoost (Tuned) Overall OOF Accuracy: 0.8034

Training RandomForest (Tuned)...
✨ RandomForest (Tuned) Overall OOF Accuracy: 0.7914

Training LightGBM (Tuned)...
✨ LightGBM (Tuned) Overall OOF Accuracy: 0.8012


### 6. SMART BLENDING & SUBMISSION

In [16]:
print("\nSmart Blending...")

# Normalize weights based on their OOF accuracy
# (Better models get slightly higher vote)
total_acc = acc_xgb + acc_rf + acc_lgb
w_xgb = acc_xgb / total_acc
w_rf = acc_rf / total_acc
w_lgb = acc_lgb / total_acc

print(f"   Weights -> XGB: {w_xgb:.2f} | RF: {w_rf:.2f} | LGB: {w_lgb:.2f}")

final_test_probs = (w_xgb * probs_xgb) + (w_rf * probs_rf) + (w_lgb * probs_lgb)
final_preds = np.argmax(final_test_probs, axis=1)

# Decode & Save
pred_labels = [TARGET_ORDER[i] for i in final_preds]

submission = pd.DataFrame({
    'RowId': df_test['RowId'],
    'AVERAGE_SPEED_DIFF': pd.Series(pred_labels).replace('None_Existent', 'None')
})

filename = '../outputs/submissions/submission_tuned_ensemble_v2.csv'
submission.to_csv(filename, index=False)
print(f"Saved to {filename}")



Smart Blending...
   Weights -> XGB: 0.34 | RF: 0.33 | LGB: 0.33
Saved to ../outputs/submissions/submission_tuned_ensemble_v2.csv
